In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N,C,H,W]: {X.shape}")
    print(f"Shape of y{y.shape} {y.dtype}")
    break




C:\Users\20385\.conda\envs\myenv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
Shape of ytorch.Size([64]) torch.int64


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
    )
print(f"Using {device} device")
#define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)
    

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataloader):
        X,y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch +1) *len(X)
            print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader,model, loss_fn)
print("Done!")
        

Epoch 1
---------------
loss:2.310808[   64/60000]
loss:2.298721[ 6464/60000]
loss:2.277423[12864/60000]
loss:2.270152[19264/60000]
loss:2.256066[25664/60000]
loss:2.231041[32064/60000]
loss:2.229250[38464/60000]
loss:2.197392[44864/60000]
loss:2.193247[51264/60000]
loss:2.173384[57664/60000]
Test Error:
 Accuracy:46.3%, Avg loss: 2.160572

Epoch 2
---------------
loss:2.172199[   64/60000]
loss:2.165326[ 6464/60000]
loss:2.102564[12864/60000]
loss:2.120900[19264/60000]
loss:2.079217[25664/60000]
loss:2.021863[32064/60000]
loss:2.043322[38464/60000]
loss:1.965039[44864/60000]
loss:1.971079[51264/60000]
loss:1.922715[57664/60000]
Test Error:
 Accuracy:56.0%, Avg loss: 1.904771

Epoch 3
---------------
loss:1.936500[   64/60000]
loss:1.915672[ 6464/60000]
loss:1.787836[12864/60000]
loss:1.833590[19264/60000]
loss:1.732914[25664/60000]
loss:1.684318[32064/60000]
loss:1.707627[38464/60000]
loss:1.599822[44864/60000]
loss:1.633070[51264/60000]
loss:1.548456[57664/60000]
Test Error:
 Accurac

In [5]:
torch.save(model.state_dict(),"model.pth")
print("Saved pyTorch Model State to model.pth")

#load model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Saved pyTorch Model State to model.pth


<All keys matched successfully>

In [6]:
print(model.parameters())

<generator object Module.parameters at 0x0000027AFF58A110>
